<a href="https://colab.research.google.com/github/ylkim1947/Car_color_classfication/blob/main/Model_description.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import cv2
import torch
import glob as glob
import os
import time

from torchvision import transforms
from torch.nn import functional as F
from torch import topk

# from model import build_model
# from class_names import class_names
class_names = ['black','red', 'white', 'blue','grey']

In [4]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

os.makedirs('/content/drive/MyDrive/cp2/codestates_cp2_cars/094/outputs/cam_results/', exist_ok=True)


In [5]:
import torch.nn as nn
from torchvision import models

def build_model(pretrained=True, fine_tune=True, num_classes=5):
    if pretrained:
        print('[INFO]: Loading pre-trained weights')
    else:
        print('[INFO]: Not loading pre-trained weights')
    #model = models.efficientnet_b1(pretrained=pretrained)
    model = models.efficientnet_v2_m(pretrained=pretrained)

    if fine_tune:
        print('[INFO]: Fine-tuning all layers...')
        for params in model.parameters():
            params.requires_grad = True
    elif not fine_tune:
        print('[INFO]: Freezing hidden layers...')
        for params in model.parameters():
            params.requires_grad = False

    # Change the final classification head.
    model.classifier[1] = nn.Linear(in_features=1280, out_features=num_classes)
    return model

In [11]:

# Define computation device.
device = 'cpu'
# Class names.
# Initialize model, switch to eval model, load trained weights.
model = build_model(
    pretrained=False,
    fine_tune=False, 
    num_classes=5
).to(device)
model = model.eval()
#print(model)
model.load_state_dict(torch.load('/content/drive/MyDrive/cp2/codestates_cp2_cars/094/outputs/model.pth',map_location=torch.device('cpu'))['model_state_dict'])

[INFO]: Not loading pre-trained weights
[INFO]: Freezing hidden layers...
EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNor

<All keys matched successfully>

In [12]:

# https://github.com/zhoubolei/CAM/blob/master/pytorch_CAM.py
def returnCAM(feature_conv, weight_softmax, class_idx):
    # Generate the class activation maps upsample to 256x256.
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

def show_cam(CAMs, width, height, orig_image, class_idx, save_name, gt_class):
    for i, cam in enumerate(CAMs):
        heatmap = cv2.applyColorMap(cv2.resize(cam,(width, height)), cv2.COLORMAP_JET)
        result = heatmap * 0.5 + orig_image * 0.5
        # put class label text on the result
        cv2.putText(result, gt_class, (20, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                    cv2.LINE_AA)
        pred_class_name = str(class_names[int(class_idx[i])])
        if pred_class_name == gt_class:
            cv2.putText(result, pred_class_name, (20, 65), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                        cv2.LINE_AA)
        else:
            cv2.putText(result, pred_class_name, (20, 65), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2,
                        cv2.LINE_AA)
        
        # cv2.imshow('CAM', result/255.)
        cv2.waitKey(1)
        cv2.imwrite(f"/content/drive/MyDrive/cp2/codestates_cp2_cars/094/outputs/cam_results/CAM_{save_name}.jpg", result)

# Hook the feature extractor.
# https://github.com/zhoubolei/CAM/blob/master/pytorch_CAM.py
features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())
model._modules.get('features').register_forward_hook(hook_feature)
# Get the softmax weight
params = list(model.parameters())
weight_softmax = np.squeeze(params[-2].data.cpu().numpy())

In [13]:
# Define the transforms, resize => tensor => normalize.
transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
        )
    ])


In [14]:
counter = 0
correct_count = 0
frame_count = 0 # To count total frames.
total_fps = 0 # To get the final frames per second. 
# Run for all the test images.
all_images = glob.glob('/content/drive/MyDrive/cp2/codestates_cp2_cars/094/test/*.jpg', recursive=True)
print(all_images[:5])
for image_path in all_images:
    # Read the image.
    image = cv2.imread(image_path)
    gt_class = image_path.split(os.path.sep)[-2]
    orig_image = image.copy()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, _ = orig_image.shape
    # Apply the image transforms.
    image_tensor = transform(image)
    # Add batch dimension.
    image_tensor = image_tensor.unsqueeze(0)
    # Forward pass through model.
    start_time = time.time()
    outputs = model(image_tensor.to(device))
    end_time = time.time()
    # Get the softmax probabilities.
    probs = F.softmax(outputs).data.squeeze()
    # Get the class indices of top k probabilities.
    class_idx = topk(probs, 1)[1].int()
    pred_class_name = str(class_names[int(class_idx)])
    if gt_class == pred_class_name:
        correct_count += 1
    
    # Generate class activation mapping for the top1 prediction.
    CAMs = returnCAM(features_blobs[0], weight_softmax, class_idx)
    # File name to save the resulting CAM image with.
    save_name = f"{image_path.split('/')[-1].split('.')[0]}"
    # Show and save the results.
    show_cam(CAMs, width, height, orig_image, class_idx, save_name, gt_class)
    counter += 1
    print(f"Image: {counter}")
    # Get the current fps.
    fps = 1 / (end_time - start_time)
    # Add `fps` to `total_fps`.
    total_fps += fps
    # Increment frame count.
    frame_count += 1

print(f"Total number of test images: {len(all_images)}")
print(f"Total correct predictions: {correct_count}")
print(f"Accuracy: {correct_count/len(all_images)*100:.3f}")

# Close all frames and video windows.
cv2.destroyAllWindows()
# calculate and print the average FPS
avg_fps = total_fps / frame_count
print(f"Average FPS: {avg_fps:.3f}")

['/content/drive/MyDrive/cp2/codestates_cp2_cars/094/test/C_210820_KI_052_19_BK_A_T_02_003.jpg', '/content/drive/MyDrive/cp2/codestates_cp2_cars/094/test/C_210807_GE_019_18_BK_A_P_01_055.jpg', '/content/drive/MyDrive/cp2/codestates_cp2_cars/094/test/C_210820_SS_017_18_BK_C_T_03_019.jpg', '/content/drive/MyDrive/cp2/codestates_cp2_cars/094/test/C_210807_GE_019_18_BK_A_P_01_038.jpg', '/content/drive/MyDrive/cp2/codestates_cp2_cars/094/test/C_210807_GE_019_18_BK_A_P_01_003.jpg']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Image: 1
Image: 2
Image: 3
Image: 4
Image: 5
Image: 6
Image: 7
Image: 8
Image: 9
Image: 10
Image: 11
Image: 12
Image: 13
Image: 14
Image: 15
Image: 16
Image: 17
Image: 18
Image: 19
Total number of test images: 19
Total correct predictions: 0
Accuracy: 0.000
Average FPS: 2.340


### torchsummary

In [15]:
!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import torchsummary 
torchsummary.summary(model,(3,480,480),batch_size=32)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 24, 240, 240]             648
       BatchNorm2d-2         [32, 24, 240, 240]              48
              SiLU-3         [32, 24, 240, 240]               0
            Conv2d-4         [32, 24, 240, 240]           5,184
       BatchNorm2d-5         [32, 24, 240, 240]              48
              SiLU-6         [32, 24, 240, 240]               0
   StochasticDepth-7         [32, 24, 240, 240]               0
       FusedMBConv-8         [32, 24, 240, 240]               0
            Conv2d-9         [32, 24, 240, 240]           5,184
      BatchNorm2d-10         [32, 24, 240, 240]              48
             SiLU-11         [32, 24, 240, 240]               0
  StochasticDepth-12         [32, 24, 240, 240]               0
      FusedMBConv-13         [32, 24, 240, 240]               0
           Conv2d-14         [32, 24, 2

### pytorch-model-summary

In [24]:
pip install pytorch-model-summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
import pytorch_model_summary
import torch

print(pytorch_model_summary.summary(model, torch.zeros(1,3,480,480),show_input=True,max_depth=0))#,batch_size=32,show_hierarchical=False))

------------------------------------------------------------------------
      Layer (type)          Input Shape         Param #     Tr. Param #
    EfficientNet-1     [1, 3, 480, 480]      52,864,761           6,405
Total params: 52,864,761
Trainable params: 6,405
Non-trainable params: 52,858,356
------------------------------------------------------------------------


### torchsummaryX

In [34]:
!pip install torchsummaryX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
import torchsummaryX
torchsummaryX.summary(model,torch.zeros((32,3,480,480)))

                                        Kernel Shape        Output Shape  \
Layer                                                                      
0_features.0.Conv2d_0                  [3, 24, 3, 3]  [32, 24, 240, 240]   
1_features.0.BatchNorm2d_1                      [24]  [32, 24, 240, 240]   
2_features.0.SiLU_2                                -  [32, 24, 240, 240]   
3_features.1.0.block.0.Conv2d_0       [24, 24, 3, 3]  [32, 24, 240, 240]   
4_features.1.0.block.0.BatchNorm2d_1            [24]  [32, 24, 240, 240]   
...                                              ...                 ...   
686_features.8.BatchNorm2d_1                  [1280]  [32, 1280, 15, 15]   
687_features.8.SiLU_2                              -  [32, 1280, 15, 15]   
688_avgpool                                        -    [32, 1280, 1, 1]   
689_classifier.Dropout_0                           -          [32, 1280]   
690_classifier.Linear_1                    [1280, 5]             [32, 5]   

           

/usr/local/lib/python3.7/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_features.0.Conv2d_0,"[3, 24, 3, 3]","[32, 24, 240, 240]",NaN,NaN
1_features.0.BatchNorm2d_1,[24],"[32, 24, 240, 240]",NaN,NaN
2_features.0.SiLU_2,-,"[32, 24, 240, 240]",NaN,NaN
3_features.1.0.block.0.Conv2d_0,"[24, 24, 3, 3]","[32, 24, 240, 240]",NaN,NaN
4_features.1.0.block.0.BatchNorm2d_1,[24],"[32, 24, 240, 240]",NaN,NaN
...,...,...,...,...
686_features.8.BatchNorm2d_1,[1280],"[32, 1280, 15, 15]",NaN,NaN
687_features.8.SiLU_2,-,"[32, 1280, 15, 15]",NaN,NaN
688_avgpool,-,"[32, 1280, 1, 1]",NaN,NaN


### torchviz

In [39]:
!pip install torchviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4151 sha256=72908a83606a2078cc943145383a1416fde4b6636426f0dbc10777d5d928dc7e
  Stored in directory: /root/.cache/pip/wheels/04/38/f5/dc4f85c3909051823df49901e72015d2d750bd26b086480ec2
Successfully built torchviz


In [43]:
from torchviz import make_dot
import torch
x= torch.zeros(32,3,480,480)
y=model(x)

make_dot(y.mean(),params=dict(model.named_parameters()),show_attrs=True,show_saved=True).render("/content/drive/MyDrive/cp2/codestates_cp2_cars/094/outputs/cam_results/efficientNetV2m.png",format='png')

'/content/drive/MyDrive/cp2/codestates_cp2_cars/094/outputs/cam_results/efficientNetV2m.png.png'

### HiddenLayer


In [45]:
!pip install hiddenlayer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
import hiddenlayer as hl 

transforms = [hl.transforms.Prune('Constant')] # removes constant nodes from graph
x= torch.zeros(32,3,480,480)
#x= torch.randn(1,8)
graph = hl.build_graph(model,x)#,transforms=transforms)
graph.theme=hl.graph.THENES['blue'].copy()
graph



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!


TypeError: ignored

In [ ]:
graph.save('efficientNetV2m_hiddenlayer',format='png')

### Netron

In [50]:
!pip install netron

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 4.9 MB/s 


In [51]:
netron -b model

SyntaxError: ignored

In [ ]:
import 